In [52]:
# import lib
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom

plt.style.use('seaborn-white')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(6278)
tf.set_random_seed(6728)
ia.seed(6278)

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

# data
mnist = input_data.read_data_sets('../../Dataset/FashionMNIST/', one_hot=False)
x_data, train_label, y_data, test_label = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
x_data_added,x_data_added_label         = mnist.validation.images,mnist.validation.labels
x_data = x_data.reshape(-1, 28, 28, 1)  # 28x28x1 input img
y_data = y_data.reshape(-1, 28, 28, 1)  # 28x28x1 input img
x_data_added = x_data_added.reshape(-1, 28, 28, 1) 

x_data      = np.vstack((x_data,x_data_added))
train_label = np.vstack((train_label[:,None],x_data_added_label[:,None]))

train_batch = np.zeros((60000,28,28,1))
test_batch  = np.zeros((10000,28,28,1))

for x in range(len(x_data)):
    train_batch[x,:,:,:] = np.expand_dims(imresize(x_data[x,:,:,0],(28,28)),axis=3)
for x in range(len(y_data)):
    test_batch[x,:,:,:] = np.expand_dims(imresize(y_data[x,:,:,0],(28,28)),axis=3)

# print out the data shape and the max and min value
print(train_batch.shape)
print(train_batch.max())
print(train_batch.min())
print(train_label.shape)
print(train_label.max())
print(train_label.min())
print(test_batch.shape)
print(test_batch.max())
print(test_batch.min())
print(test_label.shape)
print(test_label.max())
print(test_label.min())
# -- end code --

Extracting ../../Dataset/FashionMNIST/train-images-idx3-ubyte.gz
Extracting ../../Dataset/FashionMNIST/train-labels-idx1-ubyte.gz
Extracting ../../Dataset/FashionMNIST/t10k-images-idx3-ubyte.gz
Extracting ../../Dataset/FashionMNIST/t10k-labels-idx1-ubyte.gz


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning:

`imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning:

Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning:

`imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning:

Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.



(60000, 28, 28, 1)
255.0
0.0
(60000, 1)
9
0
(10000, 28, 28, 1)
255.0
0.0
(10000,)
9
0


In [39]:
# normalize and make plotly connnection
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
train_batch = train_batch/255.0
test_batch  = test_batch /255.0

In [51]:
# High level SK PCA
from sklearn.decomposition import PCA
sk_pca = PCA(2)
sk_data_fit = sk_pca.fit_transform(train_batch.reshape((60000,-1)))

color_dict = {0:'red',   1:'blue',  2:'green', 3:'yellow',4:'purple',5:'grey',  6:'black', 7:'cyan',8:'pink',  9:'skyblue'}  
color_mapping = [ color_dict[x[0]] for x in train_label ]
trace = go.Scattergl(x = sk_data_fit[:,0],y = sk_data_fit[:,1],mode = "markers",marker = dict(color=color_mapping,showscale=False))
layout= go.Layout(title=str(color_dict), xaxis={'title':'PC 1'}, yaxis={'title':'PC 2'})
figure= go.Figure(data =[trace],layout=layout)
# iplot(figure)

In [131]:
# tensorflow layers
def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater_equal(x,0),tf.float64)

def tf_iden(x)  : x
def d_tf_iden(x): x

class CNN():

    def __init__(self,k,inc,out,stddev=0.05,which_reg=0,act=tf_relu,d_act=d_tf_relu):
        self.w = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float64))
        self.m,self.v = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.act,self.d_act = act,d_act
        self.which_reg = which_reg
        
    def getw(self): return self.w

    def feedforward(self,input,stride=1,padding='SAME'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding)
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='SAME'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2

        grad = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = self.w.shape,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass = tf.nn.conv2d_backprop_input(input_sizes = [batch_size] + list(grad_part_3.shape[1:]),filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middel = learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,tf.multiply(adam_middel,m_hat)  )))
        return grad_pass,update_w
    
class tf_PCA_NN_Approach1():

    def __init__(self,inc,outc,act=tf_iden,d_act=d_tf_iden,which_reg=0.0):
        self.w        = tf.Variable(tf.random_normal([inc,outc], stddev=0.05,seed=2,dtype=tf.float64))
        self.m,self.v = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.which_reg= which_reg

    def getw(self): return self.w

    def feedforward(self,input=None):
        self.input = input
        self.layer = tf.matmul(input,self.w) 
        return self.layerA
    
    def feedforward_recon(self,input=None):
        self.input = input
        self.recon = (self.input @ self.w) @ tf.transpose(self.w)
        return self.recon

    def backprop(self,gradient=None,which_reg=0):
        grad_part_1 = gradient
        grad_part_3 = self.input
        grad_middle = grad_part_1 
        grad      = tf.matmul(tf.transpose(grad_part_3),grad_middle)/batch_size
        grad_pass = grad_middle @ self.w @ tf.transpose(self.w)
        sys.exit()

        # === Reg ===
        if self.which_reg == 0:
            grad  = grad

        if self.which_reg == 0.5:
            grad  = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)

        if self.which_reg == 1:
            grad = grad   + lamda * tf.sign(self.w)

        if self.which_reg == 1.5:
            grad = grad   + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w

        if self.which_reg == 2:
            grad = grad  + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)

        if self.which_reg == 2.5:
            grad = grad   + lamda * 2.0 * self.w

        if self.which_reg == 3:
            grad = grad   + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)

        if self.which_reg == 4:
            grad = grad   + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []

        # Update the Weight First
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1)
        v_hat = self.v / (1-beta2)
        adam_middle = m_hat *  learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle )))

        return grad_pass,update_w

In [132]:
# create classes and hyper 
learning_rate = 0.0008; batch_size = 100
beta1,beta2,adam_e = 0.9,0.999,1e-8

l1 = CNN(3,1,1)
l2 = CNN(3,1,1)
l3 = CNN(3,1,1)
PL = tf_PCA_NN_Approach1(28*28*1,2)

In [134]:
# create the place holder graph

# NOTE IN THE FRIST APPROACH 
# I am assuming the PC matrix to be orth - hence just reconstructing the data 
# by transposing the matrix!

x = tf.placeholder(shape=[batch_size,28,28,1],dtype=tf.float64)
x_flatten = tf.reshape(x,[batch_size,-1])

layer1 = l1.feedforward(x)
layer2 = l2.feedforward(layer1)
layer3 = l3.feedforward(layer2)

pc_input  = tf.reshape(layer3,[batch_size,-1])
pc_output = PL.feedforward_recon(pc_input) 

cost   = tf.reduce_mean(tf.square(pc_output-x_flatten)) * 0.5

pc_grad,pc_grad_up = PL.backprop(pc_output-x_flatten)



SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



# Reference
1. Implementing T-SNE in Tensorflow [ Manual Back Prop in TF ]. (2018). Towards Data Science. Retrieved 1 November 2018, from https://towardsdatascience.com/implementing-t-sne-in-tensorflow-manual-back-prop-in-tf-b08c21ee8e06
2. Jupyter Notebook Tutorial. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/ipython-notebook-tutorial/
3. Jupyter Notebook Tutorial. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/ipython-notebook-tutorial/
4. Offline Plots in Plotly. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/offline/
5. WebGL vs SVG. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/webgl-vs-svg/
6. Scatter Plots. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/line-and-scatter/
7. Plotly User Guide. (2018). Plot.ly. Retrieved 1 November 2018, from https://plot.ly/python/user-guide/
